## Setup


In [97]:
# !pip install Janome
# !pip install rouge-score
# !pip install bert_score --quiet
# !pip install openai --quiet

  Using cached Janome-0.5.0-py2.py3-none-any.whl (19.7 MB)

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [57]:
import os
import re
import pandas as pd
from rouge_score import rouge_scorer
from rouge_score.tokenizers import Tokenizer
from bert_score import BERTScorer

## 実行例

In [81]:
# 要約対象
# excerpt = "OpenAI's mission is to ensure that artificial general intelligence (AGI) benefits all of humanity. OpenAI will build safe and beneficial AGI directly, but will also consider its mission fulfilled if its work aids others to achieve this outcome. OpenAI follows several key principles for this purpose. First, broadly distributed benefits - any influence over AGI's deployment will be used for the benefit of all, and to avoid harmful uses or undue concentration of power. Second, long-term safety - OpenAI is committed to doing the research to make AGI safe, and to promote the adoption of such research across the AI community. Third, technical leadership - OpenAI aims to be at the forefront of AI capabilities. Fourth, a cooperative orientation - OpenAI actively cooperates with other research and policy institutions, and seeks to create a global community working together to address AGI's global challenges."

In [82]:
# 人間が作成した要約の正解例
# ref_summary = "OpenAI aims to ensure artificial general intelligence (AGI) is used for everyone's benefit, avoiding harmful uses or undue power concentration. It is committed to researching AGI safety, promoting such studies among the AI community. OpenAI seeks to lead in AI capabilities and cooperates with global research and policy institutions to address AGI's challenges."

In [83]:
# 要約結果１
# eval_summary_1 = "OpenAI aims to AGI benefits all humanity, avoiding harmful uses and power concentration. It pioneers research into safe and beneficial AGI and promotes adoption globally. OpenAI maintains technical leadership in AI while cooperating with global institutions to address AGI challenges. It seeks to lead a collaborative worldwide effort developing AGI for collective good."

In [84]:
# 要約結果2
# eval_summary_2 = "OpenAI aims to ensure AGI is for everyone's use, totally avoiding harmful stuff or big power concentration. Committed to researching AGI's safe side, promoting these studies in AI folks. OpenAI wants to be top in AI things and works with worldwide research, policy groups to figure AGI's stuff."

In [29]:
# 要約対象
excerpt="""
『葬送のフリーレン』
終幕から始まる物語…遺されたエルフは旅の中で人の心を知っていく

大勢を魅了するそのポイントは、やはり旅の中で徐々に価値観が変化していくエルフの主人公・フリーレンの姿でしょう。喪って初めて仲間の大事さを、人間という生き物の尊さを知ったフリーレン。彼女の本当の旅の始まりは、10年という短くも濃密だった冒険の後日譚から始まっていくのです。

数千年を生きるエルフを動かしたのは、たった10年の旅の記憶…

『葬送のフリーレン』あらすじ
本作の主人公・フリーレンは、長寿の種族・エルフの魔法使いです。彼女は世界を救う勇者のパーティの一員として、10年の旅の末に魔王を打ち滅ぼし世界を平和に導きました。

勇者・ヒンメルを始めとした人間たちには長い10年も、数千年を生きるエルフの彼女にとってはたった一瞬の出来事です。

旅を終えた勇者一行は解散。冒険の後は皆、それぞれに別の道を歩むこととなりました。

それから50年後。別れ際の約束通り、再び4人揃って流星群を見る事が叶った勇者一行。ですがそれから間もなく、パーティの立役者でもあった勇者・ヒンメルは大勢に惜しまれながらこの世を去りました。

彼の葬式で、10年という短い歳月だった冒険の旅を思い返すフリーレン。

その旅は彼女の長い人生にとってほんの一瞬の、取るに足らない些細な時間だったはず。ですが当時を思い返した時、彼女の中に押し寄せたのは「なぜその短い時間で、彼の事をもっと知ろうとしなかったのか」という後悔の感情でした。

自分とは種族の違う、人間という生き物の短い生と死。かつての仲間だった彼らの命に触れて、フリーレンは人間の事をもっと知りたいと思うようになります。幸いエルフである彼女には、途方もなく長い時間が残されています。勇者一行として旅を終えた彼女は、自身を主人公とした長い長い旅路を、新たにここから始めることとなったのでした。

物語は終わりから始まる…これまでにない斬新なストーリーが話題に

本作が話題となったポイントのひとつが、物語の後日譚から始まるストーリーという斬新な物語構成でしょう。

様々な異世界モノが現在もブームとなっていますが、どれだけめでたしめでたしで終わる物語でも、お話の中で生きる人々にはまだまだその先の人生が残っています。世界を救った勇者一行も、決して例外ではありません。

ある種王道でもある、異世界でラスボスを倒し故郷に戻った勇者たちの物語。本来であれば物語の終着点から始まる今作のストーリー展開は、そのあらすじだけで大勢の読者の興味を惹きつける力のあるお話ともなっていますね。

主人公らしくないフリーレンという存在から見える、人間という生き物の尊さ

加えて魅力でもあるのが、ある種主人公らしくないエルフ・フリーレンの存在です。感情の起伏に乏しく淡々としたキャラクターですが、そこにはしっかりと長命の種族ならではの達観を孕んでいます。

その諦めを抱えてなお、情動を強く揺さぶられた彼女の「人間を知りたい」という強い思いが、この物語の肝であることは言うまでもありません。

いつの時代も一定数の憧れを集める、不老不死に近い長寿。ですがその実情はフリーレンが体現するように、様々な人々から置いていかれる孤独な存在です。そんな彼女の孤独を癒すのが、もしかしたらエルフよりずっと短い時間を懸命に生きる、人間という生き物の存在なのかもしれません。

長命のエルフである彼女には当然ながら、たった数十年の寿命しかない人間の価値観を、本当の意味で理解することは難しいでしょう。

けれど、それでもフリーレンは人間を知りたいと思います。

その思いの根底にあるのは間違いなく、自分とは違う種族・人間への愛情。そして同時に、人間が持つ価値観や感情への憧れも。ないものねだりと知りながら、心のどこかで彼女は抱えているのかもしれませんね。
"""

In [30]:
# 人間が作成した要約の正解例
ref_summary = """
『葬送のフリーレン』は、エルフの魔法使いフリーレンが人間の心を理解していく旅の物語です。
かつてフリーレンは、勇者ヒンメルと共に魔王を倒し世界を救う冒険をしました。
その冒険を終えるまでに10年という月日が流れましたが、エルフの長寿からすれば人間の10年はほんの一瞬の出来事に過ぎません。
冒険の後、勇者一行は解散し、それぞれの道を歩みました。

50年後、フリーレンはヒンメルの葬式をきっかけに冒険の記憶を思い返し、なぜ彼をもっと知ろうとしなかったのだろうと後悔します。
人間の短い命と向き合った彼女は、人間をもっと知りたいと強く願うようになります。
こうしてフリーレンは再び新たな旅を始める、『葬送のフリーレン』はそんな旅を描いた作品です。

この作品は、物語の後日譚から始まる斬新な構成が話題となりました。
異世界ものの王道を踏まえつつ、冒険の終わりからその先を描くストーリー展開が多くの読者を魅了しています。

またフリーレンが持つ人間とは違った感性に、多くの人が魅了されています。
フリーレンは感情の起伏に乏しく、長命の種族ならではの達観を持っていますが、ヒンメルの死をきっかけに「人間を知りたい」という強い思いを抱くようになります。
彼女は不老不死に近い長寿の孤独を抱えながらも、人間の短い時間の中で生きる姿に魅了されます。
エルフの彼女には人間の価値観を完全に理解することは難しいものの、種族を超えた愛情と憧れを抱いています。

『葬送のフリーレン』は、エルフと人間の対比を通じた人間の尊さや感情の深さを描き出しています。
読者は、フリーレンの成長とその心情の変化、またその尊さを感じることができる作品となっています。
"""

In [31]:
# 要約結果１(gpt-4o)
eval_summary_1 = "『葬送のフリーレン』は、エルフの魔法使いフリーレンが人間の感情を理解していく旅の物語です。彼女は勇者ヒンメルと共に魔王を倒し世界を救いましたが、エルフの長寿に比べ人間の10年は一瞬の出来事です。50年後、ヒンメルの死をきっかけにフリーレンは人間の短い命に触れ、人間をもっと知りたいと願うようになります。この物語は、勇者一行の冒険の後日譚から始まり、エルフの長命と人間の短命という対比を描いています。フリーレンの変化と人間の尊さへの理解が、この作品の魅力となっています。"

# 要約結果(gpt-3.5)
eval_summary_2 = "『葬送のフリーレン』は、旅の後の物語から始まります。エルフの魔法使いである主人公・フリーレンは、勇者のパーティと共に魔王を倒し、世界を救いました。しかし、その後、彼女は10年の旅が人間にとっては一瞬でしかないことに気付きます。友人の葬儀で彼女は後悔し、人間の生と死について知りたいと思うようになります。彼女は自分の長い生と、人間の短い寿命との違いを理解しようとしますが、同時に人間の愛情や感情にも憧れます。彼女の孤独を癒すのは、人間という存在かもしれません。"

## ROUGEでの評価

In [42]:
# https://eng-blog.iij.ad.jp/archives/25669

In [99]:
tokens

<generator object Tokenizer.__tokenize_stream at 0x13bc7af80>

In [101]:
word_list

['山田', '首相', 'は', '23', '日', 'アメリカ', 'を', '訪問', 'し', 'まし', 'た']

In [106]:
class JanomeTokenizer(Tokenizer):
  """rouge-score用のJanomeを用いたTokenizerクラス
  """
  def __init__(self):
      from janome.tokenizer import Tokenizer
      self.tokenizer = Tokenizer()
      
  def tokenize(self, text):
      tokens = tokenizer.tokenize(text)
      words = [token.surface for token in tokens]
      return words

In [87]:
class TiktokenTokenizer(Tokenizer):
  """rouge-score用のTiktokenを用いたTokenizerクラス（tokenごとのN-gram）
  """
  def __init__(self):
    import tiktoken
    self.enc = tiktoken.get_encoding("cl100k_base")
      
  def tokenize(self, text):
      return self.enc.encode(text)

In [114]:
# tiktoken_tokenizer = TiktokenTokenizer()
# scorer = rouge_scorer.RougeScorer(["rouge1","rouge2"],tokenizer=tiktoken_tokenizer)
janome_tokenizer = JanomeTokenizer()
scorer = rouge_scorer.RougeScorer(["rouge1","rouge2"],tokenizer=janome_tokenizer)
rouge_score1 = scorer.score(ref_summary, eval_summary_1)
rouge_score2 = scorer.score(ref_summary, eval_summary_2)

In [111]:
# test
# print(janome_tokenizer.tokenize("山田首相は23日アメリカを訪問しました"))
# print(janome_tokenizer.tokenize("首相がアメリカを訪問した"))
# print(scorer.score("山田首相は23日アメリカを訪問しました","首相がアメリカを訪問した"))

['山田', '首相', 'は', '23', '日', 'アメリカ', 'を', '訪問', 'し', 'まし', 'た']

In [115]:
rouge_scores_out = (
    pd.DataFrame({
        "rouge1(f-score)":[
            rouge_score1["rouge1"].fmeasure,
            rouge_score2["rouge1"].fmeasure
        ],
        "rouge2(f-score)":[
            rouge_score1["rouge2"].fmeasure,
            rouge_score2["rouge2"].fmeasure
        ]
    },index=["Summary1(GPT-4o)","Summary2(GPT-3.5))"]).T.style.background_gradient(vmin=0,vmax=1)
)
rouge_scores_out

,Summary1(GPT-4o),Summary2(GPT-3.5))
rouge1(f-score),0.463122,0.375862
rouge2(f-score),0.347676,0.190311


表は、2つの異なる要約を参照テキストに対して評価したROUGEスコアを示している。rouge-1の場合、要約2が要約1を上回っており、これは個々の単語がよりよく重複していることを示している。要約2には抜粋から直接引用された多くの単語や短いフレーズがあるため、参照要約との重なりはより高く、より高いROUGEスコアにつながる可能性が高い。

ROUGEやBLEUやMETEORのような類似の測定基準は定量的な尺度を提供するが、よく作成された要約の本質を捉えられないことが多い。また、人間のスコアとの相関も悪い。流暢で首尾一貫した要約を作成することに長けているLLMの進歩を考えると、ROUGEのような従来の測定基準は、不注意にもこれらのモデルにペナルティを与えてしまう可能性がある。これは、要約の表現が異なっていても、核となる情報が正確に要約されている場合に特に当てはまる。

## BERTScoreでの評価

ROUGEは予測文と参照文の両方における単語の正確な存在に依存しており、根本的な意味論を解釈することができない。そこでBERTScoreが登場し、BERTモデルからの文脈埋め込みを活用して、機械生成テキストの文脈における予測文と参照文の類似性を評価することを目指す。両方の文の埋め込みを比較することで、BERTScoreは従来のn-gramベースのメトリクスでは見逃される可能性のある意味的類似性を捉える。

In [79]:
# BERTモデルの読み込み
bert_scorer = BERTScorer(lang="ja")

# 評価
P1, R1, F1_1 = bert_scorer.score([eval_summary_1], [ref_summary])
P2, R2, F2_2 = bert_scorer.score([eval_summary_2], [ref_summary])
print("Summary 1 F1 Score:", F1_1.tolist()[0])
print("Summary 2 F1 Score:", F2_2.tolist()[0])

Summary 1 F1 Score: 0.8276999592781067
Summary 2 F1 Score: 0.7586448788642883


両サマリーのF値/F1スコアが近いことから、重要な情報を捕捉する上で、両サマリーは同様のパフォーマンスを示している可能性がある。しかし、このわずかな差は注意して解釈すべきである。BERTScoreは、人間の評価者が理解するような微妙な点や高度な概念を完全に把握していない可能性があるため、この指標のみに依存することは、要約の実際の品質やニュアンスを誤って解釈することにつながる可能性がある。BERTScoreと人間の判断および他の評価指標を組み合わせた統合的なアプローチは、より信頼性の高い評価を提供できるだろう。

## Ada-002で評価

In [ ]:
# OpenAI APIによる埋め込みの生成
embeddings = OpenAIEmbeddings()
ref_emb = embeddings.embed_query(ref_summary)
sum_emb_1 = embeddings.embed_query(eval_summary_1)
sum_emb_2 = embeddings.embed_query(eval_summary_2)

from sklearn.metrics.pairwise import cosine_similarity
cos_sim_1 = cosine_similarity([ref_emb],[sum_emb_1])
cos_sim_2 = cosine_similarity([ref_emb],[sum_emb_2])

print("Summary 1 Ada002 Similarity:", cos_sim_1[0])
print("Summary 2 Ada002 Similarity:", cos_sim_2[0])

## GPT-4で評価

5つの観点で要約を評価：
- `正解の準備が必要`
    - **Similarity (類似性)： AIの要約と正解の要約の類似性を評価する。**
- `正解の準備が不要`
    - **Relevance (関連性)： 要約が重要な情報のみを含み、余分なものを排除しているかを評価する。（※正解の準備が）**
    - **Consistency (元文書との一貫性)： 要約が元の文書の事実と一致しているかを確認する。**
    - **Conherence (要約内容の一貫性)：要約の論理的な流れと構成を評価。要約内で矛盾した論理展開などが発生していないか、要約の構成はあやふやじゃないか、を評価する。**
    - Fluency (流暢さ)：要約の文法と可読性を評価する。

In [15]:
# Evaluation prompt template based on G-Eval
EVALUATION_PROMPT_TEMPLATE = """
You will be given one summary written for an article. Your task is to rate the summary on one metric.
Please make sure you read and understand these instructions very carefully. 
Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:
{criteria}

Evaluation Steps:
{steps}

Example:

Source Text:
{document}

Summary:
{summary}

Evaluation Form (scores ONLY):
- {metric_name}
"""

# Metric 1: Relevance
RELEVANCY_SCORE_CRITERIA = """
Relevance(1-5) - selection of important content from the source. \
The summary should include only important information from the source document. \
Annotators were instructed to penalize summaries which contained redundancies and excess information.
"""
RELEVANCY_SCORE_STEPS = """
1. Read the summary and the source document carefully.
2. Compare the summary to the source document and identify the main points of the article.
3. Assess how well the summary covers the main points of the article, and how much irrelevant or redundant information it contains.
4. Assign a relevance score from 1 to 5.
"""

# Metric 2: Coherence
COHERENCE_SCORE_CRITERIA = """
Coherence(1-5) - the collective quality of all sentences. \
We align this dimension with the DUC quality question of structure and coherence \
whereby "the summary should be well-structured and well-organized. \
The summary should not just be a heap of related information, but should build from sentence to a\
coherent body of information about a topic."
"""
COHERENCE_SCORE_STEPS = """
1. Read the article carefully and identify the main topic and key points.
2. Read the summary and compare it to the article. Check if the summary covers the main topic and key points of the article,
and if it presents them in a clear and logical order.
3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria.
"""

# Metric 3: Consistency
CONSISTENCY_SCORE_CRITERIA = """
Consistency(1-5) - the factual alignment between the summary and the summarized source. \
A factually consistent summary contains only statements that are entailed by the source document. \
Annotators were also asked to penalize summaries that contained hallucinated facts.
"""
CONSISTENCY_SCORE_STEPS = """
1. Read the article carefully and identify the main facts and details it presents.
2. Read the summary and compare it to the article. Check if the summary contains any factual errors that are not supported by the article.
3. Assign a score for consistency based on the Evaluation Criteria.
"""

# Metric 4: Fluency
FLUENCY_SCORE_CRITERIA = """
Fluency(1-3): the quality of the summary in terms of grammar, spelling, punctuation, word choice, and sentence structure.
1: Poor. The summary has many errors that make it hard to understand or sound unnatural.
2: Fair. The summary has some errors that affect the clarity or smoothness of the text, but the main points are still comprehensible.
3: Good. The summary has few or no errors and is easy to read and follow.
"""
FLUENCY_SCORE_STEPS = """
Read the summary and evaluate its fluency based on the given criteria. Assign a fluency score from 1 to 3.
"""


def get_geval_score(
    criteria: str, steps: str, document: str, summary: str, metric_name: str
):
    prompt = EVALUATION_PROMPT_TEMPLATE.format(
        criteria=criteria,
        steps=steps,
        metric_name=metric_name,
        document=document,
        summary=summary,
    )
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=5,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response.choices[0].message.content


evaluation_metrics = {
    "Relevance": (RELEVANCY_SCORE_CRITERIA, RELEVANCY_SCORE_STEPS),
    "Coherence": (COHERENCE_SCORE_CRITERIA, COHERENCE_SCORE_STEPS),
    "Consistency": (CONSISTENCY_SCORE_CRITERIA, CONSISTENCY_SCORE_STEPS),
    "Fluency": (FLUENCY_SCORE_CRITERIA, FLUENCY_SCORE_STEPS),
}

summaries = {"Summary 1": eval_summary_1, "Summary 2": eval_summary_2}

data = {"Evaluation Type": [], "Summary Type": [], "Score": []}

for eval_type, (criteria, steps) in evaluation_metrics.items():
    for summ_type, summary in summaries.items():
        data["Evaluation Type"].append(eval_type)
        data["Summary Type"].append(summ_type)
        result = get_geval_score(criteria, steps, excerpt, summary, eval_type)
        score_num = int(result.strip())
        data["Score"].append(score_num)

pivot_df = pd.DataFrame(data, index=None).pivot(
    index="Evaluation Type", columns="Summary Type", values="Score"
)
styled_pivot_df = pivot_df.style.apply(highlight_max, axis=1)
display(styled_pivot_df)

Summary Type,Summary 1,Summary 2
Evaluation Type,,
Coherence,5,3
Consistency,5,5
Fluency,3,2
Relevance,5,4


<IPython.core.display.Javascript object>

全体的に、要約1は4つのカテゴリーのうち3つ（一貫性、関連性、流暢さ）で要約2を上回っているようである。両要約は互いに一貫(要約が元の文書の事実と一致)していることがわかった。この結果は、与えられた評価基準に基づいて、要約1の方が一般的に望ましいことを示唆しているかもしれない。

### 制限事項
 LLMベースのメトリクスは、人間が書いたテキストよりもLLMが生成したテキストを好む傾向があることに注意。
 さらに、LLMベースのメトリクスは、システム・メッセージ／プロンプトの影響を受けやすい。
 私たちは、パフォーマンスを向上させ、一貫したスコアを得るのに役立つ他のテクニックを試すことを推奨する。
 また、このスコアリング手法は現在のところgpt-4のコンテキストウィンドウによって制限されていることは注目に値する。


### 結論
抽象的要約の評価は、さらなる改善の余地がある。ROUGE、BLEUScore、BERTScoreのような従来の評価基準は、有用な自動評価を提供するが、意味的類似性や要約品質の微妙な側面を捉えることには限界がある。さらに、これらは参照出力を必要とするが、その収集やラベル付けにはコストがかかる。LLMベースのメトリクスは、首尾一貫性、流暢さ、関連性を評価する参照不要の方法として有望である。しかし、これらにも、LLMによって生成されたテキストを好む潜在的なバイアスがある。結局のところ、抽象的要約システムを確実に評価するには、自動評価基準と人間による評価の組み合わせが理想的である。要約の質を包括的に理解するためには人間による評価が不可欠であるが、効率的で大規模なテストを可能にするためには、自動評価で補完されるべきである。この分野は、品質、スケーラビリティ、公平性のバランスをとりながら、より強固な評価手法を進化させ続けるだろう。評価手法を進化させることは、プロダクションアプリケーションの進歩を推進する上で極めて重要である。